In [1]:
## basic functionality
import pandas as pd
import numpy as np
import re
import os
#fuzzywuzzy packages 
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import Levenshtein
#parallel processing 
import joblib
from joblib import Parallel, delayed

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import datetime
import time
import random 
## useful material for fuzz match 
#https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings

In [2]:
df_cen = pd.read_csv("../data/census_infra_od.csv")

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\1099682092.py:1: DtypeWarning: Columns (235,286,304,312,370,371,373,374,376,377,393) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cen = pd.read_csv("../data/census_infra_od.csv")


In [3]:
df_cen.shape
#exp.shape

(51313, 396)

### Creating groups of variables
Group columns by type, manually sorted

In [4]:
       
road_cols = [col for col in df_cen.columns if ("Road" in col or 'Highway' in col)] 

fin_cols = ['ATM (Status A(1)/NA(2))',
            'Commercial Bank (Status A(1)/NA(2))',
            'Cooperative Bank (Status A(1)/NA(2))',
            'Agricultural Credit Societies (Status A(1)/NA(2))',
            'Self - Help Group (SHG) (Status A(1)/NA(2))']

power_cols = ['Power Supply For All Users Summer (April-Sept.) per day (in Hours)',
              'Power Supply For All Users Winter (Oct.-March) per day (in Hours)']

health_cols = ['Community Health Centre (Numbers)',
               'Community Health Centre Doctors Total Strength (Numbers)',
               'Community Health Centre Doctors In Position (Numbers)',
               'Community Health Centre Para Medical  Staff Total Strength (Numbers)',
               'Community Health Centre Para Medical Staff In Position (Numbers)',
               'Primary Health Centre (Numbers)',
               'Primary Health Centre  Doctors Total Strength (Numbers)', 
               'Primary Health  Centre Doctors In Position (Numbers)',
               'Primary  Health Centre Para Medical  Staff Total Strength (Numbers)',
               'Primary  Health Centre Para Medical Staff In Position (Numbers)',
               'Primary Heallth Sub Centre (Numbers)',
               'Primary Heallth Sub Centre Doctors Total Strength (Numbers)',
               'Primary Heallth Sub Centre Doctors In Position (Numbers)',
               'Primary Heallth Sub Centre Para Medical  Staff Total Strength (Numbers)',
               'Primary Heallth Sub Centre Para Medical Staff In Position (Numbers)',
               'Maternity And Child Welfare Centre (Numbers)',
               'Maternity And Child Welfare Centre Doctors Total Strength (Numbers)',
               'Maternity And Child Welfare Centre Doctors In Position (Numbers)',
               'Maternity And Child Welfare Centre Para Medical  Staff Total Strength (Numbers)',
               'Maternity And Child Welfare Centre Para Medical Staff In Position (Numbers)',
               'TB Clinic (Numbers)', 'TB Clinic Doctors Total Strength (Numbers)',
               'TB Clinic Doctors In Position (Numbers)',
               'TB Clinic Para Medical Para Medical  Staff Total Strength (Numbers)',
               'TB Clinic Para Medical Para Medical Staff In Position (Numbers)',
               'Hospital Allopathic (Numbers)', 
               'Hospital Allopathic Doctors Total Strength (Numbers)', 
               'Hospital Allopathic Doctors In Position (Numbers)',
               'Hospital Allopathic Para Medical  Staff Total Strength (Numbers)',
               'Hospital Allopathic Para Medical Staff In Position (Numbers)',
               'Hospiltal Alternative Medicine (Numbers)',
               'Hospiltal Alternative Medicine Doctors Total Strength (Numbers)',
               'Hospiltal Alternative Medicine Doctors In Position (Numbers)',
               'Hospiltal Alternative Medicine Para Medical  Staff Total Strength (Numbers)',
               'Hospiltal Alternative Medicine Para Medical Staff In Position (Numbers)',
               'Dispensary (Numbers)',
               'Dispensary Doctors Total Strength (Numbers)',
               'Dispensary Doctors In Position (Numbers)',
               'Dispensary Para Medical  Staff Total Strength (Numbers)',
               'Dispensary Para Medical Staff In Position (Numbers)',
               'Veterinary Hospital (Numbers)',
               'Veterinary Hospital Doctors Total Strength (Numbers)',
               'Veterinary Hospital Doctors In Position (Numbers)',
               'Veterinary Hospital Para Medical  Staff Total Strength (Numbers)',
               'Veterinary Hospital Para Medical Staff In Position (Numbers)',
               'Mobile Health Clinic (Numbers)', 
               'Mobile Health Clinic Doctors Total Strength (Numbers)', 
               'Mobile Health Clinic Doctors In Position (Numbers)',
               'Mobile Health Clinic Para Medical  Staff Total Strength (Numbers)',
               'Mobile Health Clinic Para Medical Staff In Position(Numbers)', 
               'Family Welfare Centre (Numbers)', 
               'Family Welfare Centre Doctors Total Strength (Numbers)',
               'Family Welfare Centre Doctors In Position (Numbers)',
               'Family Welfare Centre Para Medical  Staff Total Strength (Numbers)',
               'Family Welfare Centre Para Medical Staff In Position (Numbers)',
               'Non Government Medical facilities Out Patient (Numbers)',
               'Non Government Medical facilities In And Out Patient (Numbers)',
               'Non Government Medical facilities Charitable (Numbers)',
               'Non Government Medical facilities Medical Prctitioner with MBBS Degree (Numbers) ',
               'Non Government Medical facilities Medical Prctitioner with other Degree (Numbers)  ', 
               'Non Government Medical facilities Medical Practitioner with no  Degree (Numbers) ',
               'Non Government Medical facilities Traditional Practitioner and Faith  Healer (Numbers) ', 
               'Non Government Medical facilities Medicine Shop (Numbers) ',
               'Non Government Medical facilities Others (Numbers)']

educ_cols = ['Govt  Pre - Primary School (Nursery/LKG/UKG) (Numbers)',
             'Private Pre - Primary School (Nursery/LKG/UKG) (Numbers)', 
             'Govt Primary School (Numbers)', 
             'Private  Primary School (Numbers)',
             'Govt  Middle School (Numbers)',
             'Private Middle School (Numbers)', 
             'Govt  Secondary School (Numbers)',
             'Private Secondary  School (Numbers)', 
             'Govt Senior Secondary School (Numbers)',
             'Private Senior Secondary  School (Numbers)',
             'Govt  Arts and Science Degree College (Numbers)',
             'Private  Arts and Science Degree College (Numbers)', 
             'Govt Engineering College (Numbers)',
             'Private Engineering College (Numbers)',
             'Govt Medicine  College (Numbers)', 
             'Private Medicine College (Numbers)', 
             'Govt Management Institute (Numbers)',
             'Private Management Institute (Numbers)', 
             'Govt Polytechnic (Numbers)', 
             'Private Polytechnic (Numbers)', 
             'Govt Vocational Training School/ITI (Numbers)',
             'Private Vocational Training School/ITI (Numbers)', 
             'Government Non Formal Training Centre (Numbers)',
             'Private Non Formal Training Centre (Numbers)', 
             'Government School For Disabled (Numbers)', 
             'Private School For Disabled (Numbers)', 
             'Government Others (Numbers)', 
             'Private Others (Numbers)']

### Recoding Binary

Variables with 'Status' in column name are coded as 1 = Available, 2=Unavalaibale.

Since we will add the variables, we need binary coding (1/0).

In [5]:
# Some binary, others will be numeric

# For atm, bank variables

for col in df_cen[fin_cols]:
    df_cen[col].replace(2,0, inplace=True)

#For road vars
for col in df_cen[road_cols]:
    df_cen[col].replace(2,0, inplace=True)

ADD INDICATORS TO GENERATE SCORES

### Sum columns for amenities

two kinds of columns, numerical (continuous and binary-recoded above)

In [6]:
df_cen['fin_sum']=df_cen[fin_cols].sum(axis=1)
df_cen['roads_sum'] = df_cen[road_cols].sum(axis=1)
df_cen['power_hrs_sum'] = df_cen[power_cols].sum(axis=1)

In [7]:
df_cen['dist_cen'] = df_cen['District Name'].str.lower().str.strip()
df_cen['blk_cen'] = df_cen['CD Block Name'].str.lower().str.strip()
df_cen['gp_cen'] =df_cen['Gram Panchayat Name'].str.lower().str.strip()
df_cen['vill_cen'] = df_cen['Village Name'].str.lower().str.strip()



#### Changing district names 

Four district names were different between two variables. Rather than depending on `fuzz` match, changing them manually would yield better results during fuzz match 

In [8]:
conditions = [df_cen.dist_cen == 'baudh',df_cen.dist_cen == 'debagarh',df_cen.dist_cen == 'nabarangapur',df_cen.dist_cen == 'subarnapur']
change_with = ['boudh', 'deogarh', 'nabarangpur', 'sonepur']
df_cen['dist_cen'] = np.select(conditions,change_with,df_cen['dist_cen'])


#### Create unique IDs in census 

Add a village counter 

In [9]:
# Creating a variable for count of villages for a given ID. Since the ID is made using district+block+gp 
#and a gp has jurisdiction over multiple villages.
df_cen['id_cen'] = df_cen.dist_cen + "_" + df_cen.blk_cen + "_" + df_cen.gp_cen
df_cen['n_vill'] = 1

#### groupby census villages to GP level 


In [10]:
cen_gp_lvl = df_cen.groupby(['id_cen']).sum().reset_index()
cen_gp_lvl[['id_cen', 'Total Population of Village', 'n_vill' ]].head()

,id_cen,Total Population of Village,n_vill
0,anugul_anugul_angar bandha,4484,3
1,anugul_anugul_antulia,5648,10
2,anugul_anugul_badakantakul,6334,10
3,anugul_anugul_badakera,7259,8
4,anugul_anugul_balanga,2643,7


## Generating variables


### Demographics

- Share of SC, ST population

- total geographical area

- total population of village

- male population

- female population

- SC population

- ST population

- Nearest town distance from village


---------------------------------





In [11]:
#final list of columns we are keeping from census 
cen_final_list = ['id_cen', 
'Total Geographical Area (in Hectares)', 
'Forest Area (in Hectares)',
'Total   Households ',
'Total Population of Village',
'Total Male Population of Village',
'Total Female Population of Village',
'Total Scheduled Castes Population of Village',
'Total Scheduled Tribes Population of Village', 
'roads_sum',
'fin_sum',
'power_hrs_sum']
cen_final_list = cen_final_list + health_cols + educ_cols
cen_fnl = cen_gp_lvl[cen_final_list]

cen_fnl.head()
cen_fnl.shape
cen_fnl.columns

,id_cen,Total Geographical Area (in Hectares),Forest Area (in Hectares),Total Households,Total Population of Village,Total Male Population of Village,Total Female Population of Village,Total Scheduled Castes Population of Village,Total Scheduled Tribes Population of Village,roads_sum,...,Govt Polytechnic (Numbers),Private Polytechnic (Numbers),Govt Vocational Training School/ITI (Numbers),Private Vocational Training School/ITI (Numbers),Government Non Formal Training Centre (Numbers),Private Non Formal Training Centre (Numbers),Government School For Disabled (Numbers),Private School For Disabled (Numbers),Government Others (Numbers),Private Others (Numbers)
0,anugul_anugul_angar bandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,anugul_anugul_antulia,2620.0,855.44,1233.0,5648,2844.0,2804.0,1080.0,1940.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,anugul_anugul_badakantakul,3122.0,811.65,1509.0,6334,3235.0,3099.0,1125.0,1245.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,anugul_anugul_badakera,1533.0,130.24,1583.0,7259,3733.0,3526.0,2367.0,162.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,anugul_anugul_balanga,1358.0,236.76,614.0,2643,1304.0,1339.0,358.0,686.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(6241, 104)

Index(['id_cen', 'Total Geographical Area (in Hectares)',
       'Forest Area (in Hectares)', 'Total   Households ',
       'Total Population of Village', 'Total Male Population of Village',
       'Total Female Population of Village',
       'Total Scheduled Castes Population of Village',
       'Total Scheduled Tribes Population of Village', 'roads_sum',
       ...
       'Govt Polytechnic (Numbers)', 'Private Polytechnic (Numbers)',
       'Govt Vocational Training School/ITI (Numbers)',
       'Private Vocational Training School/ITI (Numbers)',
       'Government Non Formal Training Centre (Numbers)',
       'Private Non Formal Training Centre (Numbers)',
       'Government School For Disabled (Numbers)',
       'Private School For Disabled (Numbers)', 'Government Others (Numbers)',
       'Private Others (Numbers)'],
      dtype='object', length=104)

In [12]:
#Forest area as percentage of total geographical area 
cen_fnl['forest_perc'] = cen_fnl[ 'Forest Area (in Hectares)']/(cen_fnl['Total Geographical Area (in Hectares)'] + 1)

#Scheduled castes as percentage of total pop
cen_fnl['sc_perc'] = cen_fnl['Total Scheduled Castes Population of Village']/(cen_fnl['Total Population of Village'] + 1) 

#Scheduled Tribes 
cen_fnl['st_perc'] = cen_fnl['Total Scheduled Tribes Population of Village']/(cen_fnl['Total Population of Village'] + 1) 

#Sex ratio 
cen_fnl['fem_to_male'] = cen_fnl['Total Female Population of Village']/(cen_fnl['Total Male Population of Village'] + 1)

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\867082080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl['forest_perc'] = cen_fnl[ 'Forest Area (in Hectares)']/(cen_fnl['Total Geographical Area (in Hectares)'] + 1)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\867082080.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl['sc_perc'] = cen_fnl['Total Scheduled Castes Population of Village']/(cen_fnl['Total Population of Village'] + 1)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_3

# Cleaning expenditure data


In [13]:
exp = pd.read_csv("../data/exp_t4.csv")

#### Creating uniue IDs for expenditure data 


In [14]:
#Cleaning names
exp['blk_exp'] = exp['Block_Panchayat'].str.lower().str.strip()
exp['dist_exp']=exp['District_Panchayat'].str.lower().str.strip()
exp['gp_exp'] = exp['Village_Panchayat'].str.lower().str.strip()

#Creating expenditure ID
exp['id_exp'] = exp['dist_exp'] + "_" + exp['blk_exp'] + "_" + exp['gp_exp']

#### Group to sector level

We groupby expenditure level data by id, sector of activity and year 


In [15]:
exp['own_fund_exp']=np.where(exp.scheme_name=="Own Funds", exp['estimated_cost'], 0)

In [16]:

exp['n_activity'] = 1

exp = exp.groupby(['id_exp', 'sector', 'Plan_Year']).agg(tot_spend=('estimated_cost', np.sum),
                                               n_act = ('n_activity', np.sum),
                                               sc_fund = ('sc_fund', np.sum),
                                               st_fund = ('st_fund', np.sum),
                                               own_spend = ('own_fund_exp', np.sum)).reset_index()
                                               
                                               
exp.head()

,id_exp,sector,Plan_Year,tot_spend,n_act,sc_fund,st_fund,own_spend
0,anugul_anugul_angarbandha,Administrative & Technical Support,2015-2016,71000.0,5,0,0,0.0
1,anugul_anugul_angarbandha,Administrative & Technical Support,2016-2017,94000.0,6,0,0,0.0
2,anugul_anugul_angarbandha,Administrative & Technical Support,2017-2018,313600.0,7,0,0,0.0
3,anugul_anugul_angarbandha,Administrative & Technical Support,2018-2019,843600.0,9,0,0,0.0
4,anugul_anugul_angarbandha,Administrative & Technical Support,2019-2020,772629.0,5,0,0,0.0


## Creating district, block and gp level names from id

Since we lost them in grouping earlier 


In [17]:
#From id_cen, splitting to create dist, block and gp names 
cen_fnl[['dist_cen', 'blk_cen','gp_cen']] = cen_fnl['id_cen'].str.split('_', expand=True)

# some village names have spaces and dots, removing that
cen_fnl['id_cen'] = cen_fnl['id_cen'].str.replace(" ", "").str.replace(".", "") 

#From id_cen, splitting to create dist, block and gp names 
exp[['dist_exp', 'blk_exp','gp_exp']] = exp['id_exp'].str.split('_', expand=True)

# some village names have spaces and dots, removing that
exp['id_exp'] = exp['id_exp'].str.replace(" ", "").str.replace(".", "")

C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\2824929699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl[['dist_cen', 'blk_cen','gp_cen']] = cen_fnl['id_cen'].str.split('_', expand=True)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\2824929699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cen_fnl[['dist_cen', 'blk_cen','gp_cen']] = cen_fnl['id_cen'].str.split('_', expand=True)
C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\2824929699.py:2: SettingWithCopyWarning: 
A value is 

#### Processing for fuzzy matching 

In [18]:
#perfect matches of IDs between datasets
perfect_matches = cen_fnl['id_cen'][cen_fnl['id_cen'].isin(exp['id_exp'])]

#removing perfect matches from list of unique IDs in census data
imperfect_cen = list(set(cen_fnl['id_cen'].unique().tolist()) - set(perfect_matches.unique().tolist()))

#removing perfect matches from list of unique IDs in expenditure data
imperfect_exp = list(set(exp['id_exp'].unique().tolist()) - set(perfect_matches.unique().tolist()))

In [19]:
print("Number of unique IDs in expenditure data: " + str(len(exp['id_exp'].unique())))
print("Number of unique IDs in census data: " + str(len(cen_fnl['id_cen'].unique())))
print("Number of unique IDs that perfectly match already between two dataframes: " + str(len(perfect_matches)))


Number of unique IDs in expenditure data: 6903
Number of unique IDs in census data: 6241
Number of unique IDs that perfectly match already between two dataframes: 2997


## 2. Fuzzy merging 


2997 IDs match perfectly. There are 3906 non-matching IDs in expenditure data, and 3244 non-matching IDs in census data. 

Some GPs were newly created after the Census (from 2012 - 2021), so some GPs will not exist in the Census data.

However, due to lack of standardized spellings, there are ID strings that should match (i.e. they refer to the same entity), but do not.

Here, we use approximate string matching (or fuzzy matching). There are multiple libraries, but we use `fuzzywuzzy`.

**FuzzyWuzzy** uses Levenshtein Distance to calculate distance between patterns. There are four metrics to measure the similarity/difference between strings.

**Fuzz Ratio** - Calculate distance between ordering of tokens. We use this.

**Partial Ratio** - Take the shortest string, match with all substring of longer string

**token_sort_ratio** - Removes punctutation, sorts string alphabetically. 

**token_set_ratio** - Takes out common tokens instead of simply tokenizing. Extra words are removed.

A single ID string in the EXP data will match with multiple IDs in CEN. 
Each match pair will have its own score, indicating how similar the strings are. 

Perfect matches would get a score of 100, but we have already removed them. 

We initially keep only matches with scores > 80. This will still yield multiple candidate matches for each ID, with different scores. We only keep the match pair with the highest score.


After some manual reviewing, we find that most of the match pairs between the score of 80 - 90 are spurious.  We hence drop matches below the score of 90. (**Note: The last part is a trial and error process**)

Around 192 GPs could not even find a string similar with a score of 80 - indicates these are new names/ creations. We drop them.

In [20]:
metric = fuzz.ratio
#metric = fuzz.partial_ratio
#metric = fuzz.token_sort_ratio
#metric = fuzz.token_set_ratio

thresh = 80

ca = np.array(imperfect_exp) #array of unique IDs in expenditure data that did not have a perfect match as found above
cb = np.array(imperfect_cen) #array of unique IDs in census data that did not have a perfect match as found above



#Parallel processing for faster results
def parallel_fuzzy_match(idxa,idxb):
    return [ca[idxa],cb[idxb],metric(ca[idxa],cb[idxb])] 
results = Parallel(n_jobs=-1,verbose=1)(delayed(parallel_fuzzy_match)(idx1, idx2) for idx1 in range(len(ca)) for idx2 in range(len(cb)) \
                   if(metric(ca[idx1],cb[idx2]) > thresh))
scores = pd.DataFrame(results,columns = ["id_exp","id_cen","Score"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 464 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 2464 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 5264 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 8864 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 13264 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 16554 out of 16554 | elapsed:   43.4s finished


#### Cleaning Fuzzy scores tables

In [21]:
df_scores = scores 

#IDs in expenditure had multiple matches as long as match score was >80, therefore keeping only matchesd with best score
best_matches = df_scores.sort_values('Score', ascending = False).drop_duplicates('id_exp', keep='first') 
best_matches.reset_index().drop(columns = 'index', inplace = True)

#appending perfect matches to the above list. In this dataframe, both columns have same IDs, since they were perfect matches 
#from the beginning
perfect_match = pd.DataFrame({'id_cen':perfect_matches.unique().tolist(),
                              'id_exp':perfect_matches.unique().tolist()})
total_matches = best_matches.append(perfect_match).reset_index()
total_matches.drop(columns = 'index', inplace = True)

#filling score value for perfect matches as 100. 
total_matches = total_matches.fillna(100)

best_matches #contains only fuzzed matched IDs
best_matches.shape
total_matches # after appending perfect matches best matches 
total_matches.shape

#This shows that between 80 and 87 fuzz match is getting it wrong quite often, but it does catch some great cases. 


C:\Users\Lahiri\AppData\Local\Temp\ipykernel_36288\368570965.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_matches = best_matches.append(perfect_match).reset_index()


,id_exp,id_cen,Score
10653,jagatsinghapur_jagatsinghpur_palli,jagatsinghapur_jagatsinghapur_palli,99
135,nabarangpur_tentulikhunti_parajabarangpadar,nabarangpur_tentulikhunti_parajabarangapadar,99
10857,kendujhar_harichadanpur_bhagamunda,kendujhar_harichandanpur_bhagamunda,99
7403,kalahandi_dharamagarh_chhanchanbahali,kalahandi_dharamgarh_chhanchanbahali,99
4225,nabarangpur_nabarangpur_bhatrasiuni,nabarangpur_nabarangapur_bhatrasiuni,99
...,...,...,...
10812,balangir_khaprakhol_sahajpani,balangir_khaprakhol_khaprakhol,81
7143,nabarangpur_dabugam_anchala,nabarangpur_dabugan_chacharaguda,81
3442,sambalpur_kuchinda_kuleigarh,sambalpur_kochinda_kuntara,81
10799,balangir_agalpur_mursundhi,balangir_agalpur_kutasingh,81


(3714, 3)

,id_exp,id_cen,Score
0,jagatsinghapur_jagatsinghpur_palli,jagatsinghapur_jagatsinghapur_palli,99.0
1,nabarangpur_tentulikhunti_parajabarangpadar,nabarangpur_tentulikhunti_parajabarangapadar,99.0
2,kendujhar_harichadanpur_bhagamunda,kendujhar_harichandanpur_bhagamunda,99.0
3,kalahandi_dharamagarh_chhanchanbahali,kalahandi_dharamgarh_chhanchanbahali,99.0
4,nabarangpur_nabarangpur_bhatrasiuni,nabarangpur_nabarangapur_bhatrasiuni,99.0
...,...,...,...
6706,sundargarh_sundargarh_lankahuda,sundargarh_sundargarh_lankahuda,100.0
6707,sundargarh_sundargarh_ledhimang,sundargarh_sundargarh_ledhimang,100.0
6708,sundargarh_sundargarh_majhapara,sundargarh_sundargarh_majhapara,100.0
6709,sundargarh_sundargarh_masnikani,sundargarh_sundargarh_masnikani,100.0


(6711, 3)

In [22]:
print(str(len(exp['id_exp'].unique())-len(total_matches['id_exp'].unique()))+ 
      " IDs in the expenditure dataset whose match score with IDs in the census was less than 80, i.e. these IDs did \
 not get matched")




192 IDs in the expenditure dataset whose match score with IDs in the census was less than 80, i.e. these IDs did  not get matched


In [23]:
random.seed(27112025)

#example of why we subset for scores above 93 
#the first row should have been fuzzy matched and illustrates some data loss we face 
#due to subsetting 

total_matches[(total_matches.Score <= 89)].sample(3)

# Subset to scores >93
total_matches[(total_matches.Score >= 93)]

,id_exp,id_cen,Score
3487,gajapati_rudayagiri_dabaraguda,gajapati_rudaygiri_subalada,84.0
3350,mayurbhanj_saraskana_pandra,mayurbhanj_saraskana_sirsha,85.0
3686,ganjam_rangeilunda_laudigam,ganjam_rangeilunda_padmapur,81.0


,id_exp,id_cen,Score
0,jagatsinghapur_jagatsinghpur_palli,jagatsinghapur_jagatsinghapur_palli,99.0
1,nabarangpur_tentulikhunti_parajabarangpadar,nabarangpur_tentulikhunti_parajabarangapadar,99.0
2,kendujhar_harichadanpur_bhagamunda,kendujhar_harichandanpur_bhagamunda,99.0
3,kalahandi_dharamagarh_chhanchanbahali,kalahandi_dharamgarh_chhanchanbahali,99.0
4,nabarangpur_nabarangpur_bhatrasiuni,nabarangpur_nabarangapur_bhatrasiuni,99.0
...,...,...,...
6706,sundargarh_sundargarh_lankahuda,sundargarh_sundargarh_lankahuda,100.0
6707,sundargarh_sundargarh_ledhimang,sundargarh_sundargarh_ledhimang,100.0
6708,sundargarh_sundargarh_majhapara,sundargarh_sundargarh_majhapara,100.0
6709,sundargarh_sundargarh_masnikani,sundargarh_sundargarh_masnikani,100.0


## Merge Census + Expenditure data

In [24]:
#merging matched dataset with expenditure 
exp_matches = pd.merge(exp, total_matches, how = 'inner', left_on = 'id_exp', right_on='id_exp')
cen_fnl2= cen_fnl.drop(columns = ['dist_cen','blk_cen','gp_cen'])

#merging matched expenditure dataset with census data 
final_match = pd.merge(cen_fnl, exp_matches, how = 'inner', left_on = 'id_cen', right_on='id_cen')

In [25]:
final_match.head()
print("Shape of final merged data: " + str(final_match.shape))
print("Shape of original expenditure data: " + str(exp.shape) + " (Because this is the bigger dataset, so benchmarking against this one)")

,id_cen,Total Geographical Area (in Hectares),Forest Area (in Hectares),Total Households,Total Population of Village,Total Male Population of Village,Total Female Population of Village,Total Scheduled Castes Population of Village,Total Scheduled Tribes Population of Village,roads_sum,...,Plan_Year,tot_spend,n_act,sc_fund,st_fund,own_spend,dist_exp,blk_exp,gp_exp,Score
0,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,9.0,...,2015-2016,71000.0,5,0,0,0.0,anugul,anugul,angarbandha,100.0
1,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,9.0,...,2016-2017,94000.0,6,0,0,0.0,anugul,anugul,angarbandha,100.0
2,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,9.0,...,2017-2018,313600.0,7,0,0,0.0,anugul,anugul,angarbandha,100.0
3,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,9.0,...,2018-2019,843600.0,9,0,0,0.0,anugul,anugul,angarbandha,100.0
4,anugul_anugul_angarbandha,728.0,4.52,937.0,4484,2321.0,2163.0,461.0,152.0,9.0,...,2019-2020,772629.0,5,0,0,0.0,anugul,anugul,angarbandha,100.0


Shape of final merged data: (325225, 123)
Shape of original expenditure data: (332778, 11) (Because this is the bigger dataset, so benchmarking against this one)


In [26]:
#Without fuzzy merging, dataset size halves

merged_normal = pd.merge(exp,cen_fnl,right_on = 'id_cen', left_on = 'id_exp', how = 'inner', validate = "many_to_one")

merged_normal.shape


(149739, 122)

In [27]:
# Exporting final dataframe which will be used for analysis 
# uncomment below line to do so 
final_match.to_csv("../data/analysis_df.csv") 